In [46]:
import pandas as pd
import numpy as np
import os

In [47]:
csvfile = "./sec13f_clean_data.csv"
df = pd.read_csv(csvfile,dtype=object)

In [48]:
df.head()

,file_date,name,cusip,mval,shares
0,2013-06-30,AMERICAN EXPRESS CO,025816109,145942,1952142
1,2013-06-30,AMERICAN EXPRESS CO,025816109,1287771,17225400
2,2013-06-30,AMERICAN EXPRESS CO,025816109,62786,839832
3,2013-06-30,AMERICAN EXPRESS CO,025816109,145266,1943100
4,2013-06-30,AMERICAN EXPRESS CO,025816109,597679,7994634


In [57]:
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float

In [50]:
# create an engine to sqlite db
engine = create_engine("sqlite:///sec13f.sqlite")

In [51]:
# Create a connection to the engine called conn
conn = engine.connect()

In [52]:
# Use `declarative_base` from SQLAlchemy to connect your class to your sqlite database
Base = declarative_base()

In [53]:
class Position(Base):
    __tablename__ = 'positions'
    id = Column(Integer, primary_key=True)
    file_date = Column(Text)
    name = Column(Text)
    cusip = Column(Text)
    mval = Column(Integer)
    shares = Column (Integer)
    
    def __repr__(self):
        return f"file_date={self.file_date}, name={self.name}, cusip={self.cusip}, mval={self.mval}, shares={self.shares}"
    

In [54]:
# Use `create_all` to create the positions table in the database
Base.metadata.create_all(engine)

In [55]:
# Use MetaData from SQLAlchemy to reflect the tables\n",
metadata = MetaData(bind=engine)
metadata.reflect()

In [58]:
# Save the reference to the `positions` table as a variable called `table`
table = sqlalchemy.Table('positions', metadata, autoload=True)

In [59]:
# Use `table.delete()` to remove any existing data.
conn.execute(table.delete())

In [60]:
data = df.to_dict(orient='records')
conn.execute(table.insert(), data)

In [61]:
conn.execute("select * from positions limit 5").fetchall()

[(1, '2013-06-30', 'AMERICAN EXPRESS CO', '025816109', 145942, 1952142),
 (2, '2013-06-30', 'AMERICAN EXPRESS CO', '025816109', 1287771, 17225400),
 (3, '2013-06-30', 'AMERICAN EXPRESS CO', '025816109', 62786, 839832),
 (4, '2013-06-30', 'AMERICAN EXPRESS CO', '025816109', 145266, 1943100),
 (5, '2013-06-30', 'AMERICAN EXPRESS CO', '025816109', 597679, 7994634)]